In [57]:
using Base.Iterators
grid_world3D = zeros(10,16,5) #3D grid world in energy, H20, mission progress
actions = [[1,0,0], [0,0,1], [0,1,0], [1,1,1], [2,1,0], [1,2,0]] #possible actions, mission progress is binary
    

6-element Vector{Vector{Int64}}:
 [1, 0, 0]
 [0, 0, 1]
 [0, 1, 0]
 [1, 1, 1]
 [2, 1, 0]
 [1, 2, 0]

In [58]:
reward_model = similar(grid_world3D)
for i in 1:10, j in 1:16, k in 1:5
    if (k == 5)
        reward_model[i, j, k] = 1000
    else
        reward_model[i, j, k] = float(i)/10 + float(j)/16
    end
end    

In [60]:
#transition model
N_SS = 16*5*10
function Transition_Model(current_state, action)
    x = rand()
    if (x < 0.05)
        next_state = current_state
        reward = 0
    else 
        next_state = current_state + action
        reward = reward_model[next_state]
    end
    return next_state, reward
end

Transition_Model (generic function with 2 methods)

In [65]:
current_state = [1,1,1]
for i in 1:5
    action = actions[i]
    current_state, reward = Transition_Model(current_state, action)
    print(current_state)
end

[2, 1, 1][2, 1, 2][2, 2, 2][3, 3, 3][5, 4, 3]